In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle
import matplotlib.pyplot as plt

def load_keras_datasets():
    (x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = keras.datasets.mnist.load_data()
    (x_train_fashion, y_train_fashion), (x_test_fashion, y_test_fashion) = keras.datasets.fashion_mnist.load_data()
    (x_train_cifar10, y_train_cifar10), (x_test_cifar10, y_test_cifar10) = keras.datasets.cifar10.load_data()
    (x_train_cifar100, y_train_cifar100), (x_test_cifar100, y_test_cifar100) = keras.datasets.cifar100.load_data()
    (x_train_imdb, y_train_imdb), (x_test_imdb, y_test_imdb) = keras.datasets.imdb.load_data(num_words=10000)
    (x_train_reuters, y_train_reuters), (x_test_reuters, y_test_reuters) = keras.datasets.reuters.load_data(num_words=10000)
    
    return {
        "mnist": (x_train_mnist, y_train_mnist, x_test_mnist, y_test_mnist),
        "fashion_mnist": (x_train_fashion, y_train_fashion, x_test_fashion, y_test_fashion),
        "cifar10": (x_train_cifar10, y_train_cifar10, x_test_cifar10, y_test_cifar10),
        "cifar100": (x_train_cifar100, y_train_cifar100, x_test_cifar100, y_test_cifar100),
        "imdb": (x_train_imdb, y_train_imdb, x_test_imdb, y_test_imdb),
        "reuters": (x_train_reuters, y_train_reuters, x_test_reuters, y_test_reuters)
    }

datasets = load_keras_datasets()

def preprocess_images(x_train, x_test):
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    return x_train, x_test

for key in ["mnist", "fashion_mnist", "cifar10", "cifar100"]:
    datasets[key] = preprocess_images(datasets[key][0], datasets[key][2]) + (datasets[key][1], datasets[key][3])

def build_model(input_shape, output_shape, dropout_rate, task_type, activation):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(256, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(128, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(64, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    
    if task_type == 'classification':
        final_activation = 'softmax' if output_shape > 2 else 'sigmoid'
        loss = 'sparse_categorical_crossentropy' if output_shape > 2 else 'binary_crossentropy'
        model.add(layers.Dense(output_shape, activation=final_activation))
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    else:
        model.add(layers.Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

activations = ['relu', 'tanh', 'sigmoid', 'softmax']
dropout_rate = 0.3  
results = {}

for name, (X_train, X_test, y_train, y_test) in datasets.items():
    best_score = float('-inf')
    best_activation = None
    
    for activation in activations:
        print(f'Training {name} with activation: {activation}')
        model = build_model(X_train.shape[1:], len(np.unique(y_train)), dropout_rate, 'classification', activation)
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
        
        test_loss, test_metric = model.evaluate(X_test, y_test, verbose=0)
        results[f'{name}_{activation}'] = test_metric
        
        if test_metric > best_score:
            best_score = test_metric
            best_activation = activation
    
    print(f'Best activation for {name}: {best_activation}')

plt.figure(figsize=(10, 5))
for name in datasets.keys():
    plt.plot(activations, [results[f'{name}_{act}'] for act in activations], marker='o', label=name)
plt.title('Activation Function Effect on Model Performance')
plt.xlabel('Activation Function')
plt.ylabel('Performance Metric')
plt.legend()
plt.show()

filename = '2216052_Navya_Naveen.pkl'
with open(filename, 'wb') as f:
    pickle.dump(results, f)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.datasets import load_digits, load_iris, load_wine, load_breast_cancer, make_classification
import numpy as np
import pickle
import matplotlib.pyplot as plt

toy_datasets = {
    "digits": load_digits(return_X_y=True),
    "iris": load_iris(return_X_y=True),
    "wine": load_wine(return_X_y=True),
    "breast_cancer": load_breast_cancer(return_X_y=True)
}

datasets = {}
for name, (X, y) in toy_datasets.items():
    y = LabelEncoder().fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    datasets[name] = (X_train, X_test, y_train, y_test)

def build_model(input_shape, output_shape, dropout_rate, activation):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(256, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(128, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(64, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    
    final_activation = 'softmax' if output_shape > 2 else 'sigmoid'
    loss = 'sparse_categorical_crossentropy' if output_shape > 2 else 'binary_crossentropy'
    model.add(layers.Dense(output_shape, activation=final_activation))
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    return model

activations = ['relu', 'tanh', 'sigmoid', 'softmax']
dropout_rate = 0.3 
results = {}

for name, (X_train, X_test, y_train, y_test) in datasets.items():
    best_score = float('-inf')
    best_activation = None
    
    for activation in activations:
        print(f'Training {name} with activation: {activation}')
        model = build_model((X_train.shape[1],), len(np.unique(y_train)), dropout_rate, activation)
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
        
        test_loss, test_metric = model.evaluate(X_test, y_test, verbose=0)
        results[f'{name}_{activation}'] = test_metric
        
        if test_metric > best_score:
            best_score = test_metric
            best_activation = activation
    
    print(f'Best activation for {name}: {best_activation}')

plt.figure(figsize=(10, 5))
for name in datasets.keys():
    plt.plot(activations, [results[f'{name}_{act}'] for act in activations], marker='o', label=name)
plt.title('Activation Function Effect on Model Performance')
plt.xlabel('Activation Function')
plt.ylabel('Performance Metric')
plt.legend()
plt.show()

filename = 'toy_datasets_results.pkl'
with open(filename, 'wb') as f:
    pickle.dump(results, f)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.datasets import fetch_openml
import numpy as np
import pickle
import matplotlib.pyplot as plt

real_world_datasets = {
    "credit_g": fetch_openml(name="credit-g", as_frame=True),
    "phoneme": fetch_openml(name="phoneme", as_frame=True),
    "diabetes": fetch_openml(name="diabetes", as_frame=True),
    "blood_transfusion": fetch_openml(name="blood-transfusion-service-center", as_frame=True)
}

datasets = {}
for name, dataset in real_world_datasets.items():
    X, y = dataset.data, dataset.target
    X = X.select_dtypes(include=[np.number]).fillna(0) 
    y = LabelEncoder().fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    datasets[name] = (X_train, X_test, y_train, y_test)

def build_model(input_shape, output_shape, dropout_rate, task_type, activation):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(256, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(128, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(64, activation=activation))
    model.add(layers.Dropout(dropout_rate))
    
    if task_type == 'classification':
        final_activation = 'softmax' if output_shape > 2 else 'sigmoid'
        loss = 'sparse_categorical_crossentropy' if output_shape > 2 else 'binary_crossentropy'
        model.add(layers.Dense(output_shape, activation=final_activation))
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    else:
        model.add(layers.Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

activations = ['relu', 'tanh', 'sigmoid', 'softmax']
dropout_rate = 0.3  
results = {}

for name, (X_train, X_test, y_train, y_test) in datasets.items():
    best_score = float('-inf')
    best_activation = None
    
    for activation in activations:
        print(f'Training {name} with activation: {activation}')
        model = build_model((X_train.shape[1],), len(np.unique(y_train)), dropout_rate, 'classification', activation)
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
        
        test_loss, test_metric = model.evaluate(X_test, y_test, verbose=0)
        results[f'{name}_{activation}'] = test_metric
        
        if test_metric > best_score:
            best_score = test_metric
            best_activation = activation
    
    print(f'Best activation for {name}: {best_activation}')

plt.figure(figsize=(10, 5))
for name in datasets.keys():
    plt.plot(activations, [results[f'{name}_{act}'] for act in activations], marker='o', label=name)
plt.title('Activation Function Effect on Model Performance')
plt.xlabel('Activation Function')
plt.ylabel('Performance Metric')
plt.legend()
plt.show()

filename = '2216052_Navya_Naveen.pkl'
with open(filename, 'wb') as f:
    pickle.dump(results, f)